Antes de ejecutar esta celda se deben ejecutar los scripts `extract_utterances.py` y `textgrid_to_transcript.py` que se encuentran en la carpeta `scripts/` del dataset Primock57.

In [1]:
import os, re
from mutagen.wave import WAVE
from tools import prepare_text

basedir = '/workspace/nemo/TFM/'

# Ubicación de los WAV y las transcripciones
audio_folder = basedir + 'datasets/primock57/output/audio_utterances/'
transcript_file = basedir + 'datasets/primock57/output/transcript.ref.txt'

fout_train = open(basedir + "data_manifest/prim_train_metafile.json", "a")
fout_val = open(basedir + "data_manifest/prim_val_metafile.json", "a")
fout_test = open(basedir + "data_manifest/prim_test_metafile.json", "a")

global_count = 0
split_val = 5600
split_test = 6150
fout = fout_train
with open(transcript_file) as tfile:
    for line in tfile:
        if global_count == split_val: fout = fout_val
        if global_count == split_test: fout = fout_test
        name = re.search('\((.*)\)', line)
        audiopath = audio_folder + name.group(1) + ".wav"
        clean_line = re.sub(" \(.*\)", "", line)
        
        if os.path.exists(audiopath):
            audio = WAVE(audiopath)
            audio_length = str(audio.info.length)
        else:
            audio_length = '0'
        
        fout.write('{"audio_filepath": "' + audiopath + '", "duration": ' + audio_length + 
                   ', "text": "' + prepare_text(clean_line) + '"}\n')
        global_count += 1
        
fout_train.close()
fout_val.close()
fout_test.close()

